In [1]:
from tmu.tsetlin_machine import TMCoalescedClassifier
import numpy as np 
import pickle
import torch
from tabulate import tabulate
import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from itertools import combinations
# from apriori_python import apriori
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import seaborn as sns
!export OMP_NUM_THREADS=10
n_clauses_per_class = 8000
treshold = int(n_clauses_per_class * 0.8)
s_param = 11.40000000000001
# s_param = 5

"s henger sammen hvor ofte du ser pattern"
"lav s sier at man ser et pattern ofte, og kan glemme dette"
"belonning er s/(s-1)"
"random penalty er 1/s"

'random penalty er 1/s'

In [2]:
cuda_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0, feature_negation=False)

In [178]:
neg_cuda_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0, feature_negation=True)

In [3]:
game_amount = 100000
# tsetlin_filename, self_can_bet_index= "/home/jaoi/master22/pet_for_sale/winning_games_db/{}_tsetlined_games.pkl".format(game_amount), 17
tsetlin_filename, self_can_bet_index = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_cum_tsetlined_games.pkl".format(game_amount), 163

with open(tsetlin_filename, "rb") as fp:
        games = pickle.load(fp)

In [4]:
# preprossesing
def remove_duplicate_pairs(games):
    new_games = []
    for i,game in enumerate(games):
        int_arr = game[0].astype(int)
        complete_arr = np.append(int_arr, game[1])
        new_games.append(complete_arr)

    new_games = np.array(new_games)
    
    x = np.random.rand(new_games.shape[1])
    y = new_games.dot(x)
    unique, index = np.unique(y, return_index=True)
    unique_xy = new_games[index]
    
    unique_pairs = []
    for game in unique_xy:
        last, rest = game[-1], game[:-1]
        unique_pairs.append(np.array((rest.astype(np.float32), last), dtype=object))
    return np.array(unique_pairs)


def oversample(info_tup, desired_amount, source_games):
    bob = []
    for game in source_games:
        if game[1] == info_tup[0]:
            bob.append(game)
    indexes = np.random.randint(0, info_tup[1], desired_amount - info_tup[1])
    temp = []
    for ind in indexes:
        temp.append(bob[ind])
    return np.array(temp)


def preprocess_games(games):
    
    where_can = []
    for i, game in enumerate(games):
        if game[0][self_can_bet_index] == 1:
            where_can.append(i)
    indexes = np.array(where_can)
    print("past indexes")
    games_can_bid = games[indexes]

    unique_pairs = remove_duplicate_pairs(games_can_bid)
    print("past unique pairs")
    ysu = Counter()
    for game in unique_pairs:
        ysu[game[1]] += 1
    
    need_of_oversample = [tup for tup in ysu.most_common() if tup[1] < 2000]
    print("past unique need of oversample")
    


    unique_oversample = np.copy(unique_pairs)
    temp = []
    for overnd in need_of_oversample:
        temp.extend(oversample(overnd, 2000, unique_pairs))
    unique_oversample = np.concatenate((unique_oversample, np.array(temp)), axis=0)
    print("past unique oversample")
    
    uo = Counter()
    for game in unique_oversample:
        uo[game[1]] += 1
    print(uo)
    return unique_oversample

unique_oversample = preprocess_games(games)

past indexes
past unique pairs
past unique need of oversample
past unique oversample
Counter({4: 73819, 0: 68361, 6: 49399, 2: 43261, 3: 36861, 8: 29469, 5: 24829, 7: 15721, 9: 11462, 10: 11244, 11: 4814, 12: 3039, 1: 2000, 13: 2000, 14: 2000, 15: 2000, 16: 2000})


In [5]:
x =np.array([game[0] for game in unique_oversample[:, :1]])
y =np.array([game[0] for game in unique_oversample[:, 1:]])

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2)
# remove self bid feature, which is at index 21
x_train = np.delete(x_train, self_can_bet_index, 1)
x_test = np.delete(x_test, self_can_bet_index, 1)

In [7]:
epochs = 1
# for epoch in range(epochs):
#     tm.fit(x_train, y_train)

# print("Accuracy tm:", 100*(tm.predict(x_test) == y_test).mean())

for epoch in range(epochs):
    cuda_tm.fit(x_train, y_train)

    print("Epoch {} Accuracy tm:".format(epoch), 100*(cuda_tm.predict(x_test) == y_test).mean())

Epoch 0 Accuracy tm: 75.19226744794392


In [ ]:
epochs = 1
# for epoch in range(epochs):
#     tm.fit(x_train, y_train)

# print("Accuracy tm:", 100*(tm.predict(x_test) == y_test).mean())

for epoch in range(epochs):
    neg_cuda_tm.fit(x_train, y_train)

    print("Epoch {} Accuracy tm:".format(epoch), 100*(neg_cuda_tm.predict(x_test) == y_test).mean())

In [8]:
def get_name_of_feature_from_cum_4_rounds(feature_nr):
    feature_nr = int(feature_nr)
    money_len = 18
    score_len = 62
    order_len = 3
    stack_len = 15
    board_card_len = 15
    can_bid_len = 1
    leading_bits = [1, 2, 3]
    money_values = list(range(1, 19))
    score_values = list(range(1, 63))
    stack_values = list(range(1, 16))
    board_values = list(range(1, 16))
    
    # for self
    if feature_nr in list(range(money_len)):
        return "self coins {}".format(money_values[feature_nr])
    elif feature_nr in list(range(money_len, money_len+score_len)):
        return "self tot score {}".format(score_values[feature_nr - money_len]) 
    elif feature_nr in list(range(money_len+score_len, money_len*2 + score_len)):
        return "self bid {}".format(money_values[feature_nr - (money_len+score_len)])
    elif feature_nr in list(range(money_len*2 + score_len, money_len*2 + score_len*2)):
        return "self pass score {}".format(score_values[feature_nr - (money_len*2 + score_len)])
    elif feature_nr in list(range(money_len*2 + score_len*2, money_len*2 + score_len*2 + order_len)):
        return "self order {}".format(leading_bits[feature_nr - (money_len*2 + score_len*2)])
    
    # for next player
    elif feature_nr in list(range(money_len*2 + score_len*2 + order_len, money_len*3 + score_len*2 + order_len)):
        return "next coins {}".format(money_values[feature_nr - (money_len*2 + score_len*2 + order_len)]) 
    elif feature_nr in list(range(money_len*3 + score_len*2 + order_len, money_len*3 + score_len*3 + order_len)):
        return "next tot score {}".format(score_values[feature_nr - (money_len*3 + score_len*2 + order_len)])
    elif feature_nr in list(range(money_len*3 + score_len*3 + order_len, money_len*4 + score_len*3 + order_len)):
        return "next bid {}".format(money_values[feature_nr - (money_len*3 + score_len*3 + order_len)])
    elif feature_nr in list(range(money_len*4 + score_len*3 + order_len, money_len*4 + score_len*4 + order_len)):
        return "next pass score {}".format(score_values[feature_nr - (money_len*4 + score_len*3 + order_len)])
    elif feature_nr in list(range(money_len*4 + score_len*4 + order_len, money_len*4 + score_len*4 + order_len*2)):
        return "next order {}".format(leading_bits[feature_nr - (money_len*4 + score_len*4 + order_len)])
    elif feature_nr == (money_len*4 + score_len*4 + order_len*2):
        return "next can bid"
    
    # for final player
    elif feature_nr in list(range(money_len*4 + score_len*4 + order_len*2 + can_bid_len, money_len*5 + score_len*4 + order_len*2 + can_bid_len)):
        return "final coins {}".format(money_values[feature_nr - (money_len*4 + score_len*4 + order_len*2 + can_bid_len)])
    elif feature_nr in list(range(money_len*5 + score_len*4 + order_len*2 + can_bid_len, money_len*5 + score_len*5 + order_len*2 + can_bid_len)):
        return "final tot score {}".format(score_values[feature_nr - (money_len*5 + score_len*4 + order_len*2 + can_bid_len)])
    elif feature_nr in list(range(money_len*5 + score_len*5 + order_len*2 + can_bid_len, money_len*6 + score_len*5 + order_len*2 + can_bid_len)):
        return "final bid {}".format(money_values[feature_nr - (money_len*5 + score_len*5 + order_len*2 + can_bid_len)])
    elif feature_nr in list(range(money_len*6 + score_len*5 + order_len*2 + can_bid_len, money_len*6 + score_len*6 + order_len*2 + can_bid_len)):
        return "final pass score {}".format(score_values[feature_nr - (money_len*6 + score_len*5 + order_len*2 + can_bid_len)])
    elif feature_nr in list(range(money_len*6 + score_len*6 + order_len*2 + can_bid_len, money_len*6 + score_len*6 + order_len*3 + can_bid_len)):
        return "final order {}".format(leading_bits[feature_nr - (money_len*6 + score_len*6 + order_len*2 + can_bid_len)])
    elif feature_nr == (money_len*6 + score_len*6 + order_len*3 + can_bid_len):
        return "final can bid"
    
    # boardstuff 
    elif feature_nr in list(range(money_len*6 + score_len*6 + order_len*3 + can_bid_len*2, money_len*6 + score_len*6 + order_len*3 + stack_len + can_bid_len*2)):
        return "stack card {}".format(stack_values[feature_nr - (money_len*6 + score_len*6 + order_len*3 + can_bid_len*2)]) 
    elif feature_nr in list(range(money_len*6 + score_len*6 + order_len*3 + stack_len + can_bid_len*2, money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len + can_bid_len*2)):
        return "1. card {}".format(board_values[feature_nr - (money_len*6 + score_len*6 + order_len*3 + stack_len + can_bid_len*2)])
    elif feature_nr in list(range(money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len + can_bid_len*2, money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len*2 + can_bid_len*2)):
        return "2. card {}".format(board_values[feature_nr - (money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len + can_bid_len*2)])
    elif feature_nr in list(range(money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len*2 + can_bid_len*2, money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len*3 + can_bid_len*2)):
        return "3. card {}".format(board_values[feature_nr - (money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len*2 + can_bid_len*2)])
    elif feature_nr == money_len*6 + score_len*6 + order_len*3 + stack_len + board_card_len*3 + can_bid_len*2:
        return "std bit"
    
    

In [9]:
def show_coalesced_weight(tm, the_class, weight):
    mw = tm.weight_banks[the_class].get_weights()
    return mw[weight]


def show_coalesced_clause_4r(tm, clause):
    number_of_features = 553
    # print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(" "+get_name_of_feature_from_cum_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
    return " ∧ ".join(l)

In [10]:
def get_string_coalesced_clause_4r(tm, clause):
    number_of_features = 553
    # print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(get_name_of_feature_from_cum_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
    return l

In [11]:
def prepare_tm_for_apriori(x, y, tm, sigma=0):
    c_out = tm.transform(x.reshape(1, -1))
    clauses_out = np.where(c_out == 1)[1].astype(int)
    mw = tm.weight_banks[y].get_weights()
    weight_to_plot = mw[clauses_out]
    clauses_out_r = clauses_out[abs(weight_to_plot) > sigma]
    return clauses_out_r

In [18]:
c_out = cuda_tm.transform(x_test[0].reshape(1, -1))
clauses_out = np.where(c_out == 1)[1].astype(int)
clauses_out

array([  44,   46,  154,  225,  272,  381,  408,  466,  491,  510,  553,
        581,  856,  929,  969,  995, 1006, 1053, 1071, 1104, 1110, 1132,
       1135, 1152, 1170, 1251, 1262, 1273, 1307, 1416, 1455, 1460, 1504,
       1516, 1569, 1621, 1631, 1652, 1678, 1679, 1736, 1870, 1886, 1926,
       1947, 2020, 2051, 2059, 2112, 2140, 2294, 2321, 2532, 2591, 2665,
       2668, 2698, 2710, 2730, 2773, 2788, 2803, 2899, 2903, 2943, 2965,
       3080, 3101, 3103, 3162, 3215, 3272, 3296, 3312, 3334, 3395, 3398,
       3406, 3414, 3455, 3578, 3594, 3671, 3729, 3735, 3742, 3895, 3914,
       3932, 4050, 4102, 4115, 4264, 4343, 4363, 4378, 4425, 4452, 4545,
       4580, 4642, 4643, 4692, 4770, 4876, 4899, 4933, 4941, 5000, 5030,
       5036, 5055, 5078, 5093, 5095, 5113, 5261, 5271, 5327, 5425, 5467,
       5573, 5688, 5874, 5875, 5890, 5972, 6056, 6080, 6117, 6144, 6154,
       6159, 6177, 6184, 6226, 6364, 6394, 6416, 6421, 6525, 6535, 6616,
       6634, 6658, 6698, 6707, 6782, 6840, 6859, 68

In [19]:
show_coalesced_clause_4r(cuda_tm, 44)

' self coins 2 ∧  self tot score 2 ∧  self tot score 3 ∧  self tot score 4 ∧  self tot score 5 ∧  self tot score 7 ∧  self tot score 8 ∧  self tot score 9 ∧  self tot score 10 ∧  self tot score 12 ∧  self tot score 14 ∧  self tot score 16 ∧  self tot score 19 ∧  self tot score 20 ∧  self pass score 4 ∧  self pass score 7 ∧  self pass score 9 ∧  self pass score 10 ∧  self pass score 12 ∧  self pass score 15 ∧  self pass score 16 ∧  self pass score 18 ∧  self pass score 21 ∧  self pass score 23 ∧  self pass score 24 ∧  next coins 5 ∧  next coins 7 ∧  next coins 8 ∧  next tot score 2 ∧  next tot score 10 ∧  next tot score 11 ∧  next tot score 12 ∧  next tot score 13 ∧  next pass score 3 ∧  next pass score 4 ∧  next pass score 6 ∧  next pass score 7 ∧  next pass score 10 ∧  next pass score 15 ∧  next pass score 16 ∧  next pass score 18 ∧  next pass score 19 ∧  next pass score 20 ∧  next pass score 21 ∧  next pass score 22 ∧  final coins 1 ∧  final coins 2 ∧  final coins 4 ∧  final coins 8 

In [175]:
# oversette output cumulativ til > nummer eller < nummer
def double(lst):
    try:
        return [i+552 for i in lst]
    except TypeError:
        print(lst)


def reduce_coalesce_clause_4r(tm, clause):
    # negation
    # else:
    # l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
    own_money = list(range(18))
    high_own_money = (0, "")
    neg_own_money = (0, "")
    own_total_score = list(range(18, 80))
    high_own_total_score = (0, "")
    neg_own_total_score = (0, "")
    own_bid = list(range(80, 98))
    high_own_bid = (0, "")
    neg_own_bid = (0, "")
    own_pass_score = list(range(98, 160))
    high_own_pass_score = (0, "")
    neg_own_pass_score = (0, "")
    own_pass_order = list(range(160, 163))
    
    next_coins = list(range(163, 181))
    high_next_coins = (0, "")
    neg_next_coins = (0, "")
    next_tot_score = list(range(181, 243))
    high_next_tot_score = (0, "")
    neg_next_tot_score = (0, "")
    next_bid = list(range(243, 261))
    high_next_bid = (0, "")
    neg_next_bid = (0, "")
    next_pass_score = list(range(261, 323))
    high_next_pass_score = (0, "")
    neg_next_pass_score = (0, "")
    next_order = list(range(323, 326))
    next_can_bid = 326

    final_coins = list(range(327, 345))
    high_final_coins = (0, "")
    neg_final_coins = (0, "")
    final_tot_score = list(range(345, 407))
    high_final_tot_score = (0, "")
    neg_final_tot_score = (0, "")
    final_bid = list(range(407, 425))
    high_final_bid = (0, "")
    neg_final_bid = (0, "")
    final_pass_score = list(range(425, 487))
    high_final_pass_score = (0, "")
    neg_final_pass_score = (0, "")
    final_order = list(range(487, 490))
    final_can_bid = 490

    stack_card = list(range(491, 506))
    board_c1 = list(range(506, 521))
    high_board_c1 = (0, "")
    neg_board_c1 = (0, "")
    board_c2 = list(range(521, 536))
    high_board_c2 = (0, "")
    neg_board_c2 = (0, "")
    board_c3 = list(range(536, 551))
    high_board_c3 = (0, "")
    neg_board_c3 = (0, "")
    std_bit = 551

    
    number_of_features = 552
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1 and k in own_money:# or k in double(own_money):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_own_money[0]:
                    high_own_money = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_own_money[0]:
                    neg_own_money = (int(name[-2:]), name)
        elif tm.get_ta_action(clause, k) == 1 and k in own_total_score:# or k in double(own_total_score):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_own_total_score[0]:
                    high_own_total_score = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_own_total_score[0]:
                    neg_own_total_score = (int(name[-2:]), name)
                    
        elif tm.get_ta_action(clause, k) == 1 and k in own_bid:# or k in double(own_bid):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_own_bid[0]:
                    high_own_bid = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_own_bid[0]:
                    neg_own_bid = (int(name[-2:]), name)
        elif tm.get_ta_action(clause, k) == 1 and k in own_pass_score:# or k in double(own_pass_score):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_own_pass_score[0]:
                    high_own_pass_score = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_own_pass_score[0]:
                    neg_own_pass_score = (int(name[-2:]), name)
                    
        # find out what to do about this one!
        elif tm.get_ta_action(clause, k) == 1 and k in own_pass_order:# or k in double(own_pass_order):
            if k < number_of_features:
                l.append(get_name_of_feature_from_cum_4_rounds(k))
            # else:
            #     l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
                
                  
        elif tm.get_ta_action(clause, k) == 1 and k in next_coins:# or k in double(next_coins):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_next_coins[0]:
                    high_next_coins = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_next_coins[0]:
                    neg_next_coins = (int(name[-2:]), name)
            
        elif tm.get_ta_action(clause, k) == 1 and k in next_tot_score:# or k in double(next_tot_score):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_next_tot_score[0]:
                    high_next_tot_score = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_next_tot_score[0]:
                    neg_next_tot_score = (int(name[-2:]), name)
        elif tm.get_ta_action(clause, k) == 1 and k in next_bid:# or k in double(next_bid):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_next_bid[0]:
                    high_next_bid = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_next_bid[0]:
                    neg_next_bid = (int(name[-2:]), name)
                    
        elif tm.get_ta_action(clause, k) == 1 and k in next_pass_score:# or k in double(next_pass_score):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_next_pass_score[0]:
                    high_next_pass_score = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_next_pass_score[0]:
                    neg_next_pass_score = (int(name[-2:]), name)
                    
        elif tm.get_ta_action(clause, k) == 1 and k in next_order:# or k in double(next_order):
            if k < number_of_features:
                l.append(get_name_of_feature_from_cum_4_rounds(k))
            # else:
            #     l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
                
        elif tm.get_ta_action(clause, k) == 1 and k == next_can_bid:# or k == next_can_bid + number_of_features:
            if k < number_of_features:
                l.append(get_name_of_feature_from_cum_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))        
                
        elif tm.get_ta_action(clause, k) == 1 and k in final_coins:# or k in double(final_coins):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_final_coins[0]:
                    high_final_coins = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_final_coins[0]:
                    neg_final_coins = (int(name[-2:]), name)
        elif tm.get_ta_action(clause, k) == 1 and k in final_tot_score:# or k in double(final_tot_score):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_final_tot_score[0]:
                    high_final_tot_score = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_final_tot_score[0]:
                    neg_final_tot_score = (int(name[-2:]), name)
        elif tm.get_ta_action(clause, k) == 1 and k in final_bid:# or k in double(final_bid):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_final_bid[0]:
                    high_final_bid = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_final_bid[0]:
                    neg_final_bid = (int(name[-2:]), name)
        elif tm.get_ta_action(clause, k) == 1 and k in final_pass_score:# or k in double(final_pass_score):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_final_pass_score[0]:
                    high_final_pass_score = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_final_pass_score[0]:
                    neg_final_pass_score = (int(name[-2:]), name)
        elif tm.get_ta_action(clause, k) == 1 and k in final_order:# or k in double(final_order):
            if k < number_of_features:
                l.append(get_name_of_feature_from_cum_4_rounds(k))
            # else:
                # l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
        
        elif tm.get_ta_action(clause, k) == 1 and k == final_can_bid:# or k == final_can_bid + number_of_features:
            if k < number_of_features:
                l.append(get_name_of_feature_from_cum_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
        
        
        elif tm.get_ta_action(clause, k) == 1 and k in board_c1:# or k in double(board_c1):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_board_c1[0]:
                    high_board_c1 = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_board_c1[0]:
                    neg_board_c1 = (int(name[-2:]), name)
                    
        elif tm.get_ta_action(clause, k) == 1 and k in board_c2:# or k in double(board_c2):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_board_c2[0]:
                    high_board_c2 = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_board_c2[0]:
                    neg_board_c2 = (int(name[-2:]), name)
                    
        elif tm.get_ta_action(clause, k) == 1 and k in board_c3:# or k in double(board_c3):
            if k < number_of_features:
                name = get_name_of_feature_from_cum_4_rounds(k)
                if int(name[-2:]) > high_board_c3[0]:
                    high_board_c3 = (int(name[-2:]), name)
            else:
                name = "¬" + get_name_of_feature_from_cum_4_rounds(k - number_of_features)
                if int(name[-2:]) > neg_board_c3[0]:
                    neg_board_c3 = (int(name[-2:]), name)
                    
        elif tm.get_ta_action(clause, k) == 1 and k == std_bit:# or k == std_bit + number_of_features:
            if k < number_of_features:
                l.append(get_name_of_feature_from_cum_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
                    
        # else:
        #     if k < number_of_features:
        #         l.append(get_name_of_feature_from_cum_4_rounds(k))
            # else:
            #     l.append("¬"+get_name_of_feature_from_cum_4_rounds(k-number_of_features))
        
    
    l.append(high_own_money[1]) if high_own_money[1] != "" else None
    l.append(neg_own_money[1]) if neg_own_money[1] != "" else None
    l.append(high_own_bid[1]) if high_own_bid[1] != "" else None
    l.append(neg_own_bid[1]) if high_own_bid[1] != "" else None
    l.append(high_own_total_score[1]) if high_own_total_score[1] != "" else None
    l.append(neg_own_total_score[1]) if high_own_total_score[1] != "" else None
    l.append(high_own_pass_score[1]) if high_own_pass_score[1] != "" else None
    l.append(neg_own_pass_score[1]) if high_own_pass_score[1] != "" else None
    
    l.append(high_next_coins[1]) if high_next_coins[1] != "" else None
    l.append(neg_next_coins[1]) if high_next_coins[1] != "" else None
    l.append(high_next_tot_score[1]) if high_next_tot_score[1] != "" else None
    l.append(neg_next_tot_score[1]) if high_next_tot_score[1] != "" else None
    l.append(high_next_bid[1]) if high_next_bid[1] != "" else None
    l.append(neg_next_bid[1]) if high_next_bid[1] != "" else None
    l.append(high_next_pass_score[1]) if high_next_pass_score[1] != "" else None
    l.append(neg_next_pass_score[1]) if high_next_pass_score[1] != "" else None
    
    l.append(high_final_coins[1]) if high_final_coins[1] != "" else None
    l.append(neg_final_coins[1]) if high_final_coins[1] != "" else None
    l.append(high_final_tot_score[1]) if high_final_tot_score[1] != "" else None
    l.append(neg_final_tot_score[1]) if high_final_tot_score[1] != "" else None
    l.append(high_final_bid[1]) if high_final_bid[1] != "" else None
    l.append(neg_final_bid[1]) if high_final_bid[1] != "" else None
    l.append(high_final_pass_score[1]) if high_final_pass_score[1] != "" else None
    l.append(neg_final_pass_score[1]) if high_final_pass_score[1] != "" else None
    
    l.append(high_board_c1[1]) if high_board_c1[1] != "" else None
    l.append(neg_board_c1[1]) if high_board_c1[1] != "" else None
    l.append(high_board_c2[1]) if high_board_c2[1] != "" else None
    l.append(neg_board_c2[1]) if high_board_c2[1] != "" else None
    l.append(high_board_c3[1]) if high_board_c3[1] != "" else None
    l.append(neg_board_c3[1]) if high_board_c3[1] != "" else None
    l = [i for i in l if i != ""]
    return l

# for i in range(n_clauses_per_class):
#     if i % 1000 == 0:
#         print(i)
reduce_coalesce_clause_4r(cuda_tm, 46)


['self coins 5',
 'self tot score 18',
 'self pass score 24',
 'next coins 4',
 'next tot score 16',
 'next bid 1',
 'next pass score 24',
 'final coins 2',
 'final tot score 15',
 'final pass score 22',
 '1. card 5',
 '2. card 2',
 '3. card 1']

In [162]:
clause = 46

clause_w = [cuda_tm.weight_banks[i].get_weights()[clause] for i in range(17)]
np.argmax(clause_w)

3

In [86]:
18+62+18+62+3+18+62+18+62+3+1+18+62+18+62+3+1+15+15+15+15+1

552

In [133]:
next_pass_score = list(range(261, 323))
next_pass_score + 100

TypeError: can only concatenate list (not "int") to list

In [176]:
def single_tm_apriori(x, y, tm, sigma=-100000):
    pre_ap = prepare_tm_for_apriori(x, y, tm, sigma)
    check_clause = [reduce_coalesce_clause_4r(tm, clause) for i, clause in enumerate(pre_ap) if show_coalesced_weight(cuda_tm, y_test[20], clause) > 0]
    new_check = []
    for clause in check_clause:
        temp = []
        for feat in clause:
            temp.append(feat.strip())
        new_check.append(temp)
    te = TransactionEncoder()
    te_ary = te.fit(new_check).transform(new_check)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    return df

In [177]:
df = single_tm_apriori(x_test[0], y_test[0], cuda_tm, sigma=0)
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
pd.set_option('max_colwidth', 1000)
frequent_itemsets

,support,itemsets,length
0,0.118644,(1. card 2),1
1,0.118644,(1. card 3),1
2,0.135593,(1. card 4),1
3,0.186441,(1. card 5),1
4,0.169492,(1. card 6),1
...,...,...,...
66,0.152542,"(final tot score 14, final order 2)",2
67,0.135593,"(next order 3, final order 2)",2
68,0.118644,"(self coins 6, final order 2)",2
69,0.101695,"(self pass score 25, final order 2)",2


In [170]:
frequent_itemsets[ (frequent_itemsets['length'] == 10) &
                   (frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,length
1256190,0.847458,"(¬std bit, ¬final tot score 62, ¬next can bid, ¬final can bid, ¬self tot score 62, ¬next tot score 62, ¬final pass score 62, ¬self coins 18, ¬self pass score 62, ¬next pass score 62)",10
